In [1]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import yfinance as yf
# import pandas_datareader as web
# import datetime as dt

In [2]:
PATH = 'stocklists/'
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]

In [3]:
# sectorstocks = pd.DataFrame()
# for df in files:
#     fullist = pd.read_excel(f'stocklists/{df}')
#     sectorstocks = pd.concat([sectorstocks, fullist], axis='rows')

In [4]:
# manually find peer companies check profile on https://site.financialmodelingprep.com/profile/WMT
sectorstocks = pd.read_excel('stocklists/sectorstocks.xlsx')
pd.set_option('display.max_rows', None)
compare_companies = sectorstocks[sectorstocks['Sector'] == 'Consumer Defensive']
compare_companies = compare_companies[['Ticker','Price',
                                'Dividend Yield','1-Year Dividend Growth',
                                'Dividends Per Share (TTM)','Market Cap ($M)',
                                'Trailing P/E Ratio', 'Payout Ratio','Beta',
                               '52-Week High','52-Week Low']]
compare_companies['Market Cap ($M)'] = compare_companies['Market Cap ($M)'].round(2)
compare_companies.drop_duplicates(inplace=True)
compare_companies.sort_values(by='Market Cap ($M)',ascending=False)
# after examination I found 3 peer companies for WMT: TGT, KR, COST
# every company pays dividends
tickers = ['WMT', 'TGT', 'KR', 'COST']
compare_companies[compare_companies['Ticker'].isin(tickers)]

,Ticker,Price,Dividend Yield,1-Year Dividend Growth,Dividends Per Share (TTM),Market Cap ($M),Trailing P/E Ratio,Payout Ratio,Beta,52-Week High,52-Week Low
89,COST,480.11,0.007498,0.139241,3.481147,213038.75,36.206449,0.263125,0.933416,609.12,405.08
279,TGT,162.21,0.026632,0.200000,3.908664,74659.20,21.652901,0.531068,1.162358,249.03,134.57
316,WMT,140.54,0.015939,0.018182,2.226840,379008.28,42.267010,0.683080,0.462250,158.89,116.32
368,KR,44.45,0.023397,0.238095,0.933142,31818.28,13.493758,0.289796,0.368439,61.85,41.58


In [5]:
raw_financials = 'for_analysis/'
raw_financials_files = [x for x in listdir(raw_financials) if isfile(join(raw_financials, x))]
# merging financials to get a single datafrane for every company

# get list of files for merging
raw_financials_files 

['COST_Balance_Sheet.xlsx',
 'COST_Cash_Flow.xlsx',
 'COST_Income_Statement.xlsx',
 'KR_Balance_Sheet.xlsx',
 'KR_Cash_Flow.xlsx',
 'KR_Income_Statement.xlsx',
 'TGT_Balance_Sheet.xlsx',
 'TGT_Cash_Flow.xlsx',
 'TGT_Income_Statement.xlsx',
 'WMT_Balance_Sheet.xlsx',
 'WMT_Cash_Flow.xlsx',
 'WMT_Income_Statement.xlsx']

In [6]:
# get list of lists to merge finacials for every company
lists_for_merging = [raw_financials_files[x:x+3] for x in range(0, len(raw_financials_files), 3)]
lists_for_merging

[['COST_Balance_Sheet.xlsx',
  'COST_Cash_Flow.xlsx',
  'COST_Income_Statement.xlsx'],
 ['KR_Balance_Sheet.xlsx', 'KR_Cash_Flow.xlsx', 'KR_Income_Statement.xlsx'],
 ['TGT_Balance_Sheet.xlsx', 'TGT_Cash_Flow.xlsx', 'TGT_Income_Statement.xlsx'],
 ['WMT_Balance_Sheet.xlsx', 'WMT_Cash_Flow.xlsx', 'WMT_Income_Statement.xlsx']]

# The file is finished on 27.01.2023
# I used yearly data because to use quarterly data you need to buy subscription
# annual financials from 2022
# all formulas will be the same for quarterly data
https://site.financialmodelingprep.com/developer/docs/pricing/

In [7]:
# show all column information
pd.set_option('display.max_colwidth', None)
# merge financials for every company and save
merged_df = pd.DataFrame()
for list_of_files in lists_for_merging:
    for file in list_of_files:
        df = pd.read_excel(f'for_analysis/{file}')
        df = df.rename({'Unnamed: 0':'Index'}, axis=1)
        df = df.set_index('Index')
        merged_df = pd.concat([merged_df,df.iloc[:,0:2]])
#         merged_df = merged_df.rename({'Unnamed: 0':'Index'}, axis='columns')
#         merged_df = merged_df.set_index('Index')
        string_for_name = file.split('_')[0]
        merged_df.to_excel(f'comparables/{string_for_name}_merged.xlsx')
    merged_df = pd.DataFrame()

In [8]:
comparables = 'comparables/'
comparables_list_main = [x for x in listdir(comparables) if isfile(join(comparables, x))]
comparables_list_main

['COST_merged.xlsx', 'KR_merged.xlsx', 'TGT_merged.xlsx', 'WMT_merged.xlsx']

In [9]:
def get_investment_ratios(comparables_list):
    
    main_df = pd.DataFrame()
    for company in comparables_list:

        company_df = pd.read_excel(f'comparables/{company}', usecols = (0,1,2),index_col='Index')

        company_name = company.split('_')[0]

        stock = yf.Ticker('TGT')
        price = stock.fast_info["last_price"]

        # P/E ratio = Price per Share / Earnings per Share
        # the higher the more expensive the security
        # a couple of things regarding P/E for proper use:
        # earnings shuould be cleansed of non-recurring items
        # EPS should be normilized - Avarage ROE * BVPS(use year for EPS)
        pe_ratio = price/(company_df[company_df.columns[0]].loc['epsdiluted'])

        # P/B ratio = Price per Share / book value per share
        pb_ratio = price/(company_df[company_df.columns[0]].loc['totalStockholdersEquity'] / 
        company_df[company_df.columns[0]].loc['weightedAverageShsOutDil'])

        # P/S ratio = Price per Share / sales per share
        ps_ratio = price/(company_df[company_df.columns[0]].loc['revenue'] / 
        company_df[company_df.columns[0]].loc['weightedAverageShsOutDil'])

        # P/CF ratio = Price per share / operating cash flow per share
        pcf_ratio = price/(company_df[company_df.columns[0]].loc['operatingCashFlow'] / 
        company_df[company_df.columns[0]].loc['weightedAverageShsOutDil'])

        # EV = marketCap - cashAndCashEquivalents + totalDebt
        # ev = market value of common stock + market value of preferred equity + market value of
        # debt + minority interest – cash and investments - more precise formula from CFA
        # but we don't need it here

        ev = (price*company_df[company_df.columns[0]].loc['weightedAverageShsOutDil'] - 
        company_df[company_df.columns[0]].loc['cashAndCashEquivalents']+
        company_df[company_df.columns[0]].loc['totalDebt'])

        # EV/EBITDA
        ev_ebitda = ev/company_df[company_df.columns[0]].loc['ebitda']
        ev_ebitda

        # D/P - dividend yield = 4 * most recent quarterly dividend / price per share
        # in percent
        dp_ratio = (((-1 * company_df[company_df.columns[0]].loc['dividendsPaid'])/
                     company_df[company_df.columns[0]].loc['weightedAverageShsOutDil']) /price)

        # E/P = Earnings per Share / Price per Share
        # the higher E/P the cheaper the security
        # in percent
        ep_ratio = (company_df[company_df.columns[0]].loc['epsdiluted'])/price

        main_df_company = pd.DataFrame({company_name:[pe_ratio, pb_ratio, ps_ratio, pcf_ratio,
                                                         ev_ebitda, dp_ratio, ep_ratio]},
                              index = ['Trailing P/E', 'P/B', 'P/S', 'P/CF', 'EV/EBITDA', 
                                       'D/P(dividend yield)', 'E/P(earnings yield)'])

        main_df = pd.concat([main_df,main_df_company], axis=1)


        # the P/E ratio is not accurate due to I have yearly data, but other ratios are goood enough
    #main_df['Benchmark'] = main_df.median(axis=1)

    main_df['Benchmark'] = 0
    #https://www.investopedia.com/ask/answers/071615/what-average-pricetoearnings-ratio-retail-sector.asp
#     main_df.iloc[0,len(main_df.columns)-1] = 22.7
    main_df.iloc[0,len(main_df.columns)-1] = 'Industry norm, or relative'
    #https://eqvista.com/price-to-book-ratio-by-industry/
#     main_df.iloc[1,len(main_df.columns)-1] = 6.4
    main_df.iloc[1,len(main_df.columns)-1] = 'Industry norm, or relative'
    #https://eqvista.com/price-to-sales-ratio-by-industry/
#     main_df.iloc[2,len(main_df.columns)-1] = 3.5
    main_df.iloc[2,len(main_df.columns)-1] = 'Industry norm, or relative'
    
    main_df.iloc[3,len(main_df.columns)-1] = 'Industry norm, or relative'
    #https://www.statista.com/statistics/1030077/enterprise-value-to-ebitda-in-the-retail-and-trade-sector-worldwide/#:~:text=Worldwide%2C%20the%20average%20value%20of,from%2023.7x%20in%202020.
#     main_df.iloc[4,len(main_df.columns)-1] = 12.76
    main_df.iloc[4,len(main_df.columns)-1] = 'Industry norm, or relative'
    #https://siblisresearch.com/data/dividend-yields-sector/
#     main_df.iloc[5,len(main_df.columns)-1] = 0.0062
    main_df.iloc[5,len(main_df.columns)-1] = 'Industry norm, or relative'
    main_df.iloc[6,len(main_df.columns)-1] = 'the lower the ratio the more expensive the stock price'
    
    main_df['Desciption'] = 'info'
    main_df.iloc[0,len(main_df.columns)-1] = "how expensive the company's stock is trading at this price and EPS"
    main_df.iloc[1,len(main_df.columns)-1] = "how expensive the company's stock is trading at this price and book value of equity per share"
    main_df.iloc[2,len(main_df.columns)-1] = "how expensive the company's stock is trading at this price and sales per share"
    main_df.iloc[3,len(main_df.columns)-1] = "how expensive the company's stock is trading at this price and cash flow from operations per share"
    main_df.iloc[4,len(main_df.columns)-1] = "The enterprise multiple takes into account a company's debt and cash levels in addition to its stock price and relates that value to the firm's cash profitability"
    main_df.iloc[5,len(main_df.columns)-1] = "shows how much a company pays out in dividends each year relative to its stock price"
    main_df.iloc[6,len(main_df.columns)-1] = "the inverse of the P/E ratio, shows how much earnings per share a company generates from every dollar invested in the company’s stock"
    
    return main_df

In [10]:
def get_activity_ratios(comparables_list):
    
    main_df = pd.DataFrame()
    
    for company in comparables_list:

        company_df = pd.read_excel(f'comparables/{company}', usecols = (0,1,2),
                                   index_col='Index')
        
        company_name = company.split('_')[0]
        
        # receivables turnover = sales / average receivables
        rec_to_ratio = (company_df[company_df.columns[0]].loc['revenue'] / 
        ((company_df[company_df.columns[0]].loc['netReceivables']+
         company_df[company_df.columns[1]].loc['netReceivables'])/2))

        # days of sales outstanding = 365 / receivables turnover
        dso = 365/rec_to_ratio

        # inventory turnover = cost of goods sold / average inventory
        inv_to = (company_df[company_df.columns[0]].loc['costOfRevenue']/
        ((company_df[company_df.columns[0]].loc['inventory'][0]+
         company_df[company_df.columns[1]].loc['inventory'][0])/2))

        # days of inventory on hand = 365 / inventory turnover
        dio = 365 / inv_to

        # payables turnover = purchases / average trade payables
        # purchases =  ending inventory – beginning inventory + cost of goods sold
        pay_to = ((company_df[company_df.columns[0]].loc['inventory'][0]-
        company_df[company_df.columns[1]].loc['inventory'][0]+
        company_df[company_df.columns[0]].loc['costOfRevenue'])/
        ((company_df[company_df.columns[0]].loc['accountPayables']+
        company_df[company_df.columns[1]].loc['accountPayables'])/2))

        # number of days of payables = 365 / payables turnover 
        dpo = 365 / pay_to

        # total asset turnover = revenue / average total assets
        tot_ast_to = (company_df[company_df.columns[0]].loc['revenue']/ 
        ((company_df[company_df.columns[0]].loc['totalAssets']+
        company_df[company_df.columns[1]].loc['totalAssets'])/2))

        # fixed asset turnover = revenue / average net fixed assets
        fix_ast_to = (company_df[company_df.columns[0]].loc['revenue']/ 
        ((company_df[company_df.columns[0]].loc['propertyPlantEquipmentNet']+
        company_df[company_df.columns[1]].loc['propertyPlantEquipmentNet'])/2))

        # working capital turnover = revenue / average working capital
        work_cap_to = (company_df[company_df.columns[0]].loc['revenue']/
        (((company_df[company_df.columns[0]].loc['totalCurrentAssets']-
        company_df[company_df.columns[0]].loc['totalCurrentLiabilities'])+
        (company_df[company_df.columns[1]].loc['totalCurrentAssets']-
        company_df[company_df.columns[1]].loc['totalCurrentLiabilities']))/2))

        main_df_company = pd.DataFrame({company_name:[rec_to_ratio, dso, inv_to, dio, pay_to, dpo, 
                                                      tot_ast_to, fix_ast_to, work_cap_to]},
                                       index = ['Receivables turnover', 'Days of sales outstanding', 
                                                'Inventory turnover', 'Days of inventory on hand', 
                                                'Payables turnover', 'Number of days of payables', 
                                                'Total asset turnover','Fixed asset turnover',
                                                'Working capital turnover'])

        main_df = pd.concat([main_df,main_df_company], axis=1)
        
    # for norms use this site https://csimarket.com/Industry/industry_Efficiency.php?s=400
    main_df['Benchmark'] = 0
    main_df.iloc[0,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[1,len(main_df.columns)-1] = '<40'
    #https://www.shopify.com/retail/inventory-turnover-ratio
#     main_df.iloc[2,len(main_df.columns)-1] = 8.8
    main_df.iloc[2,len(main_df.columns)-1] = 'Industry norm, or relative'
    #https://www.investopedia.com/terms/d/days-sales-inventory-dsi.asp
    main_df.iloc[3,len(main_df.columns)-1] = 'between 30 and 60'
    #https://www.bdc.ca/en/articles-tools/entrepreneur-toolkit/templates-business-guides/glossary/payables-turnover
    main_df.iloc[4,len(main_df.columns)-1] = 'between 8 and 10'
    main_df.iloc[5,len(main_df.columns)-1] = 'the less the better, but not always'
    #https://csimarket.com/Industry/industry_Efficiency.php?s=1300
#     main_df.iloc[6,len(main_df.columns)-1] = 1.91
    main_df.iloc[6,len(main_df.columns)-1] = 'Industry norm, or relative'
    main_df.iloc[7,len(main_df.columns)-1] = 'the higher the better, but not always'
    main_df.iloc[8,len(main_df.columns)-1] = 'the higher the better, but not always'
    
    main_df['Desciption'] = 'info'
    main_df.iloc[0,len(main_df.columns)-1] = 'measure used to quantify how efficiently a company is in collecting receivables from its clients'
    main_df.iloc[1,len(main_df.columns)-1] = 'the average number of days it takes a company to receive payment for a sale'
    main_df.iloc[2,len(main_df.columns)-1] = 'the number of times a business sells and replaces its stock of goods during a given period'
    main_df.iloc[3,len(main_df.columns)-1] = 'how many days it takes a business to sell through their stock of inventory'
    main_df.iloc[4,len(main_df.columns)-1] = 'the rate at which a company pays off its suppliers'
    main_df.iloc[5,len(main_df.columns)-1] = 'the average number of days it takes a company to pay back its accounts payable'
    main_df.iloc[6,len(main_df.columns)-1] = 'an indicator of the efficiency with which a company is using its assets to generate revenue'
    main_df.iloc[7,len(main_df.columns)-1] = 'how well or efficiently a business uses fixed assets to generate sales'
    main_df.iloc[8,len(main_df.columns)-1] = 'measures how effective a business is at generating sales for every dollar of working capital put to use'
    
    return main_df

In [11]:
def get_liquidity_ratios(comparables_list):
    main_df = pd.DataFrame()
    
    for company in comparables_list:

        company_df = pd.read_excel(f'comparables/{company}', usecols = (0,1,2),
                                   index_col='Index')
        
        company_name = company.split('_')[0]
        
        # current ratio = current assets / current liabilities
        cur_ratio = (company_df[company_df.columns[0]].loc['totalCurrentAssets']/
        company_df[company_df.columns[0]].loc['totalCurrentLiabilities'])

        # quick ratio = (cash + marketable securities + receivables) / current liabilities
        qk_ratio = ((company_df[company_df.columns[0]].loc['cashAndCashEquivalents']+ 
        company_df[company_df.columns[0]].loc['shortTermInvestments']+
        company_df[company_df.columns[0]].loc['netReceivables'])/
        company_df[company_df.columns[0]].loc['totalCurrentLiabilities'])

        # cash ratio = (cash + marketable securities) / current liabilities
        cash_ratio = ((company_df[company_df.columns[0]].loc['cashAndCashEquivalents']+ 
        company_df[company_df.columns[0]].loc['shortTermInvestments'])/
        company_df[company_df.columns[0]].loc['totalCurrentLiabilities'])

        # operating cycle = dso + dio

        rec_to_ratio = (company_df[company_df.columns[0]].loc['revenue'] / 
        ((company_df[company_df.columns[0]].loc['netReceivables']+
        company_df[company_df.columns[1]].loc['netReceivables'])/2))

        dso = 365/rec_to_ratio

        inv_to = (company_df[company_df.columns[0]].loc['costOfRevenue']/
        ((company_df[company_df.columns[0]].loc['inventory'][0]+
        company_df[company_df.columns[1]].loc['inventory'][0])/2))

        dio = 365 / inv_to

        pay_to = ((company_df[company_df.columns[0]].loc['inventory'][0]-
        company_df[company_df.columns[1]].loc['inventory'][0]+
        company_df[company_df.columns[0]].loc['costOfRevenue'])/
        ((company_df[company_df.columns[0]].loc['accountPayables']+
        company_df[company_df.columns[1]].loc['accountPayables'])/2))

        dpo = 365 / pay_to

        oper_cycle = dso + dio

        # cash conversion cycle = dso + dio - dpo
        cc_cycle = dso + dio - dpo

        main_df_company = pd.DataFrame({company_name:[cur_ratio, qk_ratio, cash_ratio, oper_cycle, 
                                            cc_cycle]},
                                       index = ['Current Ratio', 'Quick Ratio', 
                                                'Cash Ratio', 'Operating Cycle', 
                                                'Cash Conversion Cycle'])

        main_df = pd.concat([main_df,main_df_company], axis=1)
# for norms use this site https://csimarket.com/Industry/industry_Efficiency.php?s=400

    main_df['Benchmark'] = 0
    #https://www.businessinsider.com/personal-finance/current-ratio
    main_df.iloc[0,len(main_df.columns)-1] = 'between 1.5 and 3'
    #https://csimarket.com/Industry/industry_Financial_Strength_Ratios.php?s=1300
#     main_df.iloc[1,len(main_df.columns)-1] = 0.3
    main_df.iloc[1,len(main_df.columns)-1] = 'Industry norm, or relative'
    #https://corporatefinanceinstitute.com/resources/accounting/cash-ratio-formula/
    main_df.iloc[2,len(main_df.columns)-1] = 'between 0.5 and 1'
    #https://www.linkedin.com/pulse/understanding-companys-cash-conversion-cycle-uriah-j-bisson-mba/
    main_df.iloc[3,len(main_df.columns)-1] = 'Industry norm'
    main_df.iloc[4,len(main_df.columns)-1] = 'the shorter the better'
    
    main_df['Desciption'] = 'info'
    main_df.iloc[0,len(main_df.columns)-1] = "measures a company's ability to pay short-term obligations or those due within one year"
    main_df.iloc[1,len(main_df.columns)-1] = 'measures a company’s ability to meet its short-term obligations with its most liquid assets'
    main_df.iloc[2,len(main_df.columns)-1] = 'the number of times a business sells and replaces its stock of goods during a given period'
    main_df.iloc[3,len(main_df.columns)-1] = 'how many days it takes a business to sell through their stock of inventory'
    main_df.iloc[4,len(main_df.columns)-1] = 'the rate at which a company pays off its suppliers'

    return main_df

In [12]:
def get_solvency_ratios(comparables_list):
    
    main_df = pd.DataFrame()
    
    for company in comparables_list:

        company_name = company.split('_')[0]

        company_df = pd.read_excel(f'comparables/{company}', usecols = (0,1,2),
                                   index_col='Index')

        # debt to assets = total debt / total assets
        d_asst = (company_df[company_df.columns[0]].loc['totalDebt']/
        company_df[company_df.columns[0]].loc['totalAssets'])

        # debt to equity = total debt / total equity
        d_eqt = (company_df[company_df.columns[0]].loc['totalDebt']/
        company_df[company_df.columns[0]].loc['totalEquity'])

        # debt to capital  = total tebt / (total debt + shareholder's equity)
        d_cap = (company_df[company_df.columns[0]].loc['totalDebt']/
        (company_df[company_df.columns[0]].loc['totalDebt']+ 
        company_df[company_df.columns[0]].loc['totalStockholdersEquity']))

        # financial leverage = average total assets / average total equity
        fin_lev = (((company_df[company_df.columns[0]].loc['totalAssets']+ 
        company_df[company_df.columns[1]].loc['totalAssets'])/2)/
        ((company_df[company_df.columns[0]].loc['totalEquity']+ 
        company_df[company_df.columns[1]].loc['totalEquity'])/2))

        # interest coverage = ebit / interest payments
        int_cov = (company_df[company_df.columns[0]].loc['operatingIncome']/
        company_df[company_df.columns[0]].loc['interestExpense'])
        int_cov

        # long term debt to capitalization = long term debt / (long term debt + shareholder's equity)
        ltd_tocap = (company_df[company_df.columns[0]].loc['longTermDebt']/
        (company_df[company_df.columns[0]].loc['longTermDebt']+ 
        company_df[company_df.columns[0]].loc['totalStockholdersEquity'])) 

        main_df_company = pd.DataFrame({company_name:[d_asst, d_eqt, d_cap, fin_lev, 
                                            int_cov, ltd_tocap]},
                                       index = ['Debt-to-Asset', 'Debt-to-Equity', 
                                                'Debt-to-Capital', 'Financial Leverage',
                                                'Interest Coverage', 'Long term Debt-to-Capital'])

        main_df = pd.concat([main_df,main_df_company], axis=1)
        
    main_df['Benchmark'] = 0
    main_df.iloc[0,len(main_df.columns)-1] = 'the higher the more the debt'
    main_df.iloc[1,len(main_df.columns)-1] = 'the higher the more the debt'
    main_df.iloc[2,len(main_df.columns)-1] = 'the higher the more the debt'
    main_df.iloc[3,len(main_df.columns)-1] = 'the higher the more the riskier'
    main_df.iloc[4,len(main_df.columns)-1] = 'the higher the more the easier'
    main_df.iloc[5,len(main_df.columns)-1] = 'the higher the more the easier'
    
    main_df['Desciption'] = 'info'
    main_df.iloc[0,len(main_df.columns)-1] = "shows how much of a business is owned by creditors (people it has borrowed money from) compared with how much of the company's assets are owned by shareholders"
    main_df.iloc[1,len(main_df.columns)-1] = 'highlights how a company’s capital structure is tilted either toward debt or equity financing'
    main_df.iloc[2,len(main_df.columns)-1] = 'how much debt a company uses to fund its operational and functional costs compared to using its capital. Capital comprises the assets and cash a business has. The purpose of the D/C ratio is to measure the amount of risk a company takes regarding debt and overall financial operation'
    main_df.iloc[3,len(main_df.columns)-1] = 'the use of borrowed money (debt) to finance the purchase of assets with the expectation that the income or capital gain from the new asset will exceed the cost of borrowing'
    main_df.iloc[4,len(main_df.columns)-1] = 'how easily a company can pay interest on its outstanding debt'
    main_df.iloc[5,len(main_df.columns)-1] = 'how much financial leverage a firm has and whether its main source of funding comes from debts'

    return main_df

In [13]:
def get_profitability_ratios(comparables_list):
    main_df = pd.DataFrame()
    
    for company in comparables_list:

        company_name = company.split('_')[0]

        company_df = pd.read_excel(f'comparables/{company}', usecols = (0,1,2),
                                           index_col='Index')

        # net profit margin = net income / revenue
        netpr_mar = (company_df[company_df.columns[0]].loc['netIncome'][1]/
        company_df[company_df.columns[0]].loc['revenue'])

        # gross profit margin = gross profit / revenue
        grpr_mar = (company_df[company_df.columns[0]].loc['grossProfit']/
        company_df[company_df.columns[0]].loc['revenue'])

        # operating profit margin = ebit / revenue
        oppr_mar = (company_df[company_df.columns[0]].loc['operatingIncome']/
        company_df[company_df.columns[0]].loc['revenue'])

        # return on assets = net income / average total assets
        roa = (company_df[company_df.columns[0]].loc['netIncome'][1]/
        ((company_df[company_df.columns[0]].loc['totalAssets']+ 
        company_df[company_df.columns[1]].loc['totalAssets'])/2))

        # return on equity = net income / average total equity
        roe = (company_df[company_df.columns[0]].loc['netIncome'][1]/
            ((company_df[company_df.columns[0]].loc['totalEquity']+ 
        company_df[company_df.columns[1]].loc['totalEquity'])/2))

        # return on total capital(roce) = ebit / average total capital
        rotc = (company_df[company_df.columns[0]].loc['operatingIncome']/      
        (((company_df[company_df.columns[0]].loc['totalAssets']+ 
        company_df[company_df.columns[1]].loc['totalAssets'])/2)-      
        ((company_df[company_df.columns[0]].loc['totalCurrentLiabilities']+
        company_df[company_df.columns[1]].loc['totalCurrentLiabilities'])/2)))

        main_df_company = pd.DataFrame({company_name:[netpr_mar, grpr_mar, oppr_mar, roa, 
                                                    roe, rotc]},
                                               index = ['Net profit margin', 
                                                        'Gross profit margin', 
                                                        'Operating profit margin', 
                                                        'Return on assets', 
                                                        'Return on equity', 
                                                        'Return on total capital'])

        main_df = pd.concat([main_df,main_df_company], axis=1)
        
    main_df['Benchmark'] = 0
    main_df.iloc[0,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[1,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[2,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[3,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[4,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[5,len(main_df.columns)-1] = 'the higher the better'
    
    main_df['Desciption'] = 'info'
    main_df.iloc[0,len(main_df.columns)-1] = "how much net income or profit is generated as a percentage of revenue"
    main_df.iloc[1,len(main_df.columns)-1] = 'what your business made after paying for the direct cost of doing business, which can include labour, materials and other direct production costs'
    main_df.iloc[2,len(main_df.columns)-1] = "how well your company's operations contribute to its profitability"
    main_df.iloc[3,len(main_df.columns)-1] = 'how much profit a company can generate from its assets'
    main_df.iloc[4,len(main_df.columns)-1] = "efficient a company's management is at generating income and growth from its equity financing"
    main_df.iloc[5,len(main_df.columns)-1] = 'quantifies how much return a company has generated through the use of its capital structure'
    
    return main_df

In [14]:
def get_performance_ratios(comparables_list):
    main_df = pd.DataFrame()
    
    for company in comparables_list:

        company_name = company.split('_')[0]

        company_df = pd.read_excel(f'comparables/{company}', usecols = (0,1,2),
                                           index_col='Index')

        # cash flow to revenue = CFO / revenue
        cf_torev = (company_df[company_df.columns[0]].loc['netCashProvidedByOperatingActivities']/
        company_df[company_df.columns[0]].loc['revenue'])

        # cash return on assets = CFO / average total assets
        cf_toast = (company_df[company_df.columns[0]].loc['netCashProvidedByOperatingActivities']/
        ((company_df[company_df.columns[0]].loc['totalAssets']+ 
        company_df[company_df.columns[1]].loc['totalAssets'])/2))


        # cash return on equity = CFO / average total equity
        cf_toeq = (company_df[company_df.columns[0]].loc['netCashProvidedByOperatingActivities']/
        ((company_df[company_df.columns[0]].loc['totalEquity']+ 
        company_df[company_df.columns[1]].loc['totalEquity'])/2))


        main_df_company = pd.DataFrame({company_name:[cf_torev, cf_toast, cf_toeq]},
                                               index = ['Cash flow to revenue', 
                                                        'Cash return on assets', 
                                                        'Cash return on equity'])

        main_df = pd.concat([main_df,main_df_company], axis=1)
        
    main_df['Benchmark'] = 0
    main_df.iloc[0,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[1,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[2,len(main_df.columns)-1] = 'the higher the better'

    main_df['Desciption'] = 'info'
    main_df.iloc[0,len(main_df.columns)-1] = "the money a company earns from the sale of its products and services"
    main_df.iloc[1,len(main_df.columns)-1] = 'how efficiently a company uses Assets to generate Operating Cashflows'
    main_df.iloc[2,len(main_df.columns)-1] = "how much cash flow seems to one dollar of invested capital"

    return main_df

In [15]:
def get_coverage_ratios(comparables_list):
    main_df = pd.DataFrame()
    
    for company in comparables_list:

        company_name = company.split('_')[0]

        company_df = pd.read_excel(f'comparables/{company}', usecols = (0,1,2),
                                           index_col='Index')

        # debt coverage = CFO / total debt
        d_cov = (company_df[company_df.columns[0]].loc['netCashProvidedByOperatingActivities']/
        company_df[company_df.columns[0]].loc['totalDebt'])

        # capex coverage = CFO / CAPEX
        cap_cov = (company_df[company_df.columns[0]].loc['netCashProvidedByOperatingActivities']/
        ((-1)*company_df[company_df.columns[0]].loc['capitalExpenditure']))

        # interest coverage = (CFO + interest paid + taxes paid)/ interest paid
        int_cov = ((company_df[company_df.columns[0]].loc['netCashProvidedByOperatingActivities']+
        company_df[company_df.columns[0]].loc['interestExpense']+
        company_df[company_df.columns[0]].loc['incomeTaxExpense'])/
        company_df[company_df.columns[0]].loc['interestExpense'])

        # dividend coverage = CFO / dividends paid
        div_cov = (company_df[company_df.columns[0]].loc['netCashProvidedByOperatingActivities']/
        ((-1)*company_df[company_df.columns[0]].loc['dividendsPaid']))
        
        main_df_company = pd.DataFrame({company_name:[d_cov, cap_cov, int_cov, div_cov]},
                                               index = ['Debt coverage', 'Capex coverage',
                                                        'Interest coverage','Dividend coverage'])

        main_df = pd.concat([main_df,main_df_company], axis=1)
        # for norms use this site https://csimarket.com/Industry/industry_Efficiency.php?s=400
        
    main_df['Benchmark'] = 0
    main_df.iloc[0,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[1,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[2,len(main_df.columns)-1] = 'the higher the better'
    main_df.iloc[3,len(main_df.columns)-1] = 'the higher the better'

    main_df['Desciption'] = 'info'
    main_df.iloc[0,len(main_df.columns)-1] = "cash flow available to pay current debt obligations"
    main_df.iloc[1,len(main_df.columns)-1] = 'company’s ability to generate enough cash from daily operations to cover capital expenditures'
    main_df.iloc[2,len(main_df.columns)-1] = "measures the firm’s ability to meet its interest obligations"
    main_df.iloc[3,len(main_df.columns)-1] = "how many times a company’s dividends can be paid using its net income"
    
    return main_df

In [16]:
# company = pd.read_excel('comparables/KR_merged.xlsx', usecols = (0,1,2),index_col='Index')
# company

In [17]:
get_investment_ratios(comparables_list_main)

,COST,KR,TGT,WMT,Benchmark,Desciption
Trailing P/E,12.711568,80.690821,11.846099,34.297741,"Industry norm, or relative",how expensive the company's stock is trading at this price and EPS
P/B,3.598865,14.225471,6.415817,5.627655,"Industry norm, or relative",how expensive the company's stock is trading at this price and book value of equity per share
P/S,0.327325,0.975133,0.776338,0.818011,"Industry norm, or relative",how expensive the company's stock is trading at this price and sales per share
P/CF,10.049751,21.721995,9.541528,19.375507,"Industry norm, or relative",how expensive the company's stock is trading at this price and cash flow from operations per share
EV/EBITDA,7.441107,25.344064,7.735813,16.443555,"Industry norm, or relative",The enterprise multiple takes into account a company's debt and cash levels in addition to its stock price and relates that value to the firm's cash profitability
D/P(dividend yield),0.020165,0.004381,0.018810,0.013131,"Industry norm, or relative",shows how much a company pays out in dividends each year relative to its stock price
E/P(earnings yield),0.078669,0.012393,0.084416,0.029156,the lower the ratio the more expensive the stock price,"the inverse of the P/E ratio, shows how much earnings per share a company generates from every dollar invested in the company’s stock"


In [18]:
get_activity_ratios(comparables_list_main)

,COST,KR,TGT,WMT,Benchmark,Desciption
Receivables turnover,112.242334,78.035088,186.792952,77.420114,the higher the better,measure used to quantify how efficiently a company is in collecting receivables from its clients
Days of sales outstanding,3.251892,4.677383,1.954035,4.714537,<40,the average number of days it takes a company to receive payment for a sale
Inventory turnover,12.414046,15.510060,6.105722,8.456535,"Industry norm, or relative",the number of times a business sells and replaces its stock of goods during a given period
Days of inventory on hand,29.402178,23.533114,59.779991,43.161888,between 30 and 60,how many days it takes a business to sell through their stock of inventory
Payables turnover,11.901424,15.927224,5.520133,8.439723,between 8 and 10,the rate at which a company pays off its suppliers
Number of days of payables,30.668599,22.916737,66.121599,43.247863,"the less the better, but not always",the average number of days it takes a company to pay back its accounts payable
Total asset turnover,3.677334,2.923152,2.018009,2.303195,"Industry norm, or relative",an indicator of the efficiency with which a company is using its assets to generate revenue
Fixed asset turnover,8.436638,4.660819,3.542770,5.148999,"the higher the better, but not always",how well or efficiently a business uses fixed assets to generate sales
Working capital turnover,595.679790,-36.760331,463.916849,-128.897041,"the higher the better, but not always",measures how effective a business is at generating sales for every dollar of working capital put to use


In [19]:
get_liquidity_ratios(comparables_list_main)

,COST,KR,TGT,WMT,Benchmark,Desciption
Current Ratio,1.021814,0.745819,0.991999,0.927797,between 1.5 and 3,measures a company's ability to pay short-term obligations or those due within one year
Quick Ratio,0.415338,0.289836,0.271808,0.263679,"Industry norm, or relative",measures a company’s ability to meet its short-term obligations with its most liquid assets
Cash Ratio,0.345303,0.177847,0.271808,0.168919,between 0.5 and 1,the number of times a business sells and replaces its stock of goods during a given period
Operating Cycle,32.654070,28.210497,61.734027,47.876425,Industry norm,how many days it takes a business to sell through their stock of inventory
Cash Conversion Cycle,1.985472,5.293760,-4.387572,4.628562,the shorter the better,the rate at which a company pays off its suppliers


In [20]:
get_solvency_ratios(comparables_list_main)

,COST,KR,TGT,WMT,Benchmark,Desciption
Debt-to-Asset,0.140869,0.416412,0.301295,0.234105,the higher the more the debt,shows how much of a business is owned by creditors (people it has borrowed money from) compared with how much of the company's assets are owned by shareholders
Debt-to-Equity,0.437894,2.162505,1.263974,0.688540,the higher the more the debt,highlights how a company’s capital structure is tilted either toward debt or equity financing
Debt-to-Capital,0.304538,0.683795,0.558299,0.407772,the higher the more the debt,how much debt a company uses to fund its operational and functional costs compared to using its capital. Capital comprises the assets and cash a business has. The purpose of the D/C ratio is to measure the amount of risk a company takes regarding debt and overall financial operation
Financial Leverage,3.230749,5.225546,3.852972,3.029371,the higher the more the riskier,the use of borrowed money (debt) to finance the purchase of assets with the expectation that the income or capital gain from the new asset will exceed the cost of borrowing
Interest Coverage,49.322785,6.089317,21.249406,13.010030,the higher the more the easier,how easily a company can pay interest on its outstanding debt
Long term Debt-to-Capital,0.302824,0.670513,0.555683,0.384992,the higher the more the easier,how much financial leverage a firm has and whether its main source of funding comes from debts


In [21]:
get_profitability_ratios(comparables_list_main)

,COST,KR,TGT,WMT,Benchmark,Desciption
Net profit margin,0.025750,0.012002,0.065525,0.023872,the higher the better,how much net income or profit is generated as a percentage of revenue
Gross profit margin,0.121487,0.220099,0.292835,0.250987,the higher the better,"what your business made after paying for the direct cost of doing business, which can include labour, materials and other direct production costs"
Operating profit margin,0.034337,0.025216,0.084392,0.045293,the higher the better,how well your company's operations contribute to its profitability
Return on assets,0.094690,0.035085,0.132230,0.054983,the higher the better,how much profit a company can generate from its assets
Return on equity,0.305921,0.183339,0.509480,0.166563,the higher the better,efficient a company's management is at generating income and growth from its equity financing
Return on total capital,0.251407,0.109038,0.283160,0.163501,the higher the better,quantifies how much return a company has generated through the use of its capital structure


In [22]:
get_performance_ratios(comparables_list_main)

,COST,KR,TGT,WMT,Benchmark,Desciption
Cash flow to revenue,0.032570,0.044892,0.081364,0.042219,the higher the better,the money a company earns from the sale of its products and services
Cash return on assets,0.119773,0.131225,0.164193,0.097238,the higher the better,how efficiently a company uses Assets to generate Operating Cashflows
Cash return on equity,0.386955,0.685721,0.632633,0.294571,the higher the better,how much cash flow seems to one dollar of invested capital


In [23]:
get_coverage_ratios(comparables_list_main)

,COST,KR,TGT,WMT,Benchmark,Desciption
Debt coverage,0.817790,0.302838,0.531981,0.421838,the higher the better,cash flow available to pay current debt obligations
Capex coverage,1.899769,2.368018,2.433691,1.845033,the higher the better,company’s ability to generate enough cash from daily operations to cover capital expenditures
Interest coverage,59.968354,12.514886,26.144893,15.512036,the higher the better,measures the firm’s ability to meet its interest obligations
Dividend coverage,4.934579,10.509338,5.571705,3.930592,the higher the better,how many times a company’s dividends can be paid using its net income
